# Fantasy Points - Calculation
---

### 1. Libraries Import

In [59]:
import warnings
import pandas as pd
from pathlib import Path
import json
import requests

In [60]:
# Suppress the specific warning about data validation
warnings.filterwarnings("ignore", category=UserWarning, message="Data Validation extension is not supported")

# Load the data from the "Fantasy" sheet of the Excel file
excel_path = Path("resources/HMFC_2023-1.xlsx")
sheet_name = "Fantasy"
df = pd.read_excel(excel_path, sheet_name, index_col="Player Full Name")
df.fillna(0, inplace=True)
df.index = df.index.str.replace(',', '')

In [61]:
df.head()

,League,Season,Team,Position,Result,Jersey,Hodler Mark Points,Minutes Played,Goal,Goal Assist,Yellow Card,Red Card,Goal Against,Penalties Won,Penalties Converted,Penalties Committed,Penalties Saved,Penalties Missed,Free-kick Converted
Player Full Name,,,,,,,,,,,,,,,,,,,
Julio Villamar,PFL_Division_1,2023_Spring,Hodler Miami FC,DEF,Lost,11,4,90,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0
Christian Lugo,PFL_Division_1,2023_Spring,Hodler Miami FC,DEF,Lost,2,3,60,0.0,0.0,1.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0
Stephen Fonseca,PFL_Division_1,2023_Spring,Hodler Miami FC,DEF,Lost,3,3,90,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0
Lucas Eceizabarrena,PFL_Division_1,2023_Spring,Hodler Miami FC,FWD,Lost,15,3,80,0.0,0.0,1.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0
Alexis Perez,PFL_Division_1,2023_Spring,Hodler Miami FC,MID,Lost,6,4,90,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
def calculate_fantasy_points(row):
    points = 0

    # Minutes Played
    if row["Minutes Played"] < 60:
        points += 1
    else:
        points += 2

    # Goal points based on Position
    if not pd.isna(row["Goal"]):
        if row["Position"] == "GOA" or row["Position"] == "DEF":
            points += 6
        elif row["Position"] == "MID":
            points += 5
        elif row["Position"] == "STR":
            points += 4

    # Goal Assist
    if not pd.isna(row["Goal Assist"]):
        points += 3

    # Clean Sheet points
    if row["Position"] == "GOA" and row["Minutes Played"] > 60:
        points += 4
    elif row["Position"] == "DEF" and row["Minutes Played"] > 60:
        points += 3
    elif row["Position"] == "MID" and row["Minutes Played"] > 60:
        points += 2
    elif row["Position"] == "STR" and row["Minutes Played"] > 60:
        points += 1

    # Missed Penalty
    if not pd.isna(row["Penalties Missed"]):
        points -= 2

    # Saved Penalty
    if not pd.isna(row["Penalties Saved"]):
        points += 5

    # Provoked Penalty
    if row["Penalties Won"] > 0:
        points += 2

    # Penalty Committed
    if not pd.isna(row["Penalties Committed"]):
        points -= 2

    # Conceded Goals points
    if (row["Position"] == "GOA" or row["Position"] == "DEF") and row["Goal Against"] > 0:
        points -= row["Goal Against"] // 2 * 2
    elif (row["Position"] == "MID" or row["Position"] == "STR") and row["Goal Against"] > 0:
        points -= row["Goal Against"] // 2

    # Yellow Card
    if not pd.isna(row["Yellow Card"]):
        points -= 1

    # Second Yellow Card
    if not pd.isna(row["Red Card"]) and row["Red Card"] == "Second Yellow Card":
        points -= 1

    # Red Card
    if not pd.isna(row["Red Card"]) and row["Red Card"] == "Red Card":
        points -= 3
        
    # Hodler points:
    points += row["Hodler Mark Points"]

    return points

# Apply the function to each row of the DataFrame
df["Fantasy Points"] = df.apply(calculate_fantasy_points, axis=1)

df.head()

,Fantasy Points,League,Season,Team,Position,Result,Jersey,Hodler Mark Points,Minutes Played,Goal,Goal Assist,Yellow Card,Red Card,Goal Against,Penalties Won,Penalties Converted,Penalties Committed,Penalties Saved,Penalties Missed,Free-kick Converted
Player Full Name,,,,,,,,,,,,,,,,,,,,
Julio Villamar,16,PFL_Division_1,2023_Spring,Hodler Miami FC,DEF,Lost,11,4,90,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0
Christian Lugo,12,PFL_Division_1,2023_Spring,Hodler Miami FC,DEF,Lost,2,3,60,0.0,0.0,1.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0
Stephen Fonseca,15,PFL_Division_1,2023_Spring,Hodler Miami FC,DEF,Lost,3,3,90,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0
Lucas Eceizabarrena,8,PFL_Division_1,2023_Spring,Hodler Miami FC,FWD,Lost,15,3,80,0.0,0.0,1.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0
Alexis Perez,15,PFL_Division_1,2023_Spring,Hodler Miami FC,MID,Lost,6,4,90,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
cols = list(df.columns)
cols.remove("Fantasy Points")
cols.insert(0, "Fantasy Points")
df = df[cols]

df.head()

,Fantasy Points,League,Season,Team,Position,Result,Jersey,Hodler Mark Points,Minutes Played,Goal,Goal Assist,Yellow Card,Red Card,Goal Against,Penalties Won,Penalties Converted,Penalties Committed,Penalties Saved,Penalties Missed,Free-kick Converted
Player Full Name,,,,,,,,,,,,,,,,,,,,
Julio Villamar,16,PFL_Division_1,2023_Spring,Hodler Miami FC,DEF,Lost,11,4,90,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0
Christian Lugo,12,PFL_Division_1,2023_Spring,Hodler Miami FC,DEF,Lost,2,3,60,0.0,0.0,1.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0
Stephen Fonseca,15,PFL_Division_1,2023_Spring,Hodler Miami FC,DEF,Lost,3,3,90,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0
Lucas Eceizabarrena,8,PFL_Division_1,2023_Spring,Hodler Miami FC,FWD,Lost,15,3,80,0.0,0.0,1.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0
Alexis Perez,15,PFL_Division_1,2023_Spring,Hodler Miami FC,MID,Lost,6,4,90,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
json_data = {}
for player_name, player_data in df.iterrows():
    player_dict = player_data.to_dict()
    json_data[player_name] = player_dict

formatted_json = json.dumps(json_data, indent=4)

file_path = 'hodlerfc.json'
with open(file_path, 'w') as json_file:
    json_file.write(formatted_json)